In [1]:
import findspark
findspark.init()
findspark.find()

import pyspark
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession

### Resilient Distributed Datasets (RDDs)

Spark revolves around the concept of a resilient distributed dataset (RDD), which is a fault-tolerant collection of elements that can be operated on in parallel.
There are two ways to create RDDs: 
1. parallelizing an existing collection in your driver program
2. referencing a dataset in an external storage system, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.

#### Parallelized Collections

- Parallelized collections are created by calling SparkContext’s parallelize method on an existing iterable or collection in your driver program.
-  The elements of the collection are copied to form a distributed dataset that can be operated on in parallel.

In [2]:
# Creating a SparkContext object

sc=SparkContext()

In [3]:
data=[1,2,3,4,5]
distData=sc.parallelize(data)

In [4]:
distData.collect()

[1, 2, 3, 4, 5]

In [5]:
type(distData)

pyspark.rdd.RDD

In [6]:
distData.reduce(lambda x,y:x+y)

15

One important parameter for parallel collections is the number of partitions to cut the dataset into. Spark will run one task for each partition of the cluster. Typically you want 2-4 partitions for each CPU in your cluster. Normally, Spark tries to set the number of partitions automatically based on your cluster. However, you can also set it manually by passing it as a second parameter to parallelize (e.g. sc.parallelize(data, 10)). Note: some places in the code use the term slices (a synonym for partitions) to maintain backward compatibility.

#### External Datasets

- PySpark can create distributed datasets from any storage source supported by Hadoop, including your local file system, HDFS, Cassandra, HBase, Amazon S3, etc. Spark supports text files, SequenceFiles, and any other Hadoop InputFormat.
- Text file RDDs can be created using SparkContext’s textFile method. This method takes a URI for the file (either a local path on the machine, or a hdfs://, s3a://, etc URI) and reads it as a collection of lines. 

In [7]:
%%writefile example.txt
first 
second line
the third line
then a fourth line

Overwriting example.txt


In [8]:
sc.textFile('example.txt').collect()

['first ', 'second line', 'the third line', 'then a fourth line']

- All transformations in Spark are lazy, in that they do not compute their results right away. Instead, they just remember the transformations applied to some base dataset 
-  The transformations are only computed when an action requires a result to be returned to the driver program.This design enables Spark to run more efficiently.

In [9]:
rdd=sc.parallelize(range(1,4)).map(lambda x:(x, 'a'*x))
rdd.collect()

[(1, 'a'), (2, 'aa'), (3, 'aaa')]

In [10]:
lines=sc.textFile('example.txt')
lines.collect()

['first ', 'second line', 'the third line', 'then a fourth line']

In [11]:
lineLenghts=lines.map(lambda s:len(s))
print(lineLenghts.collect())
totalLength=lineLenghts.reduce(lambda a,b:a+b)
print(totalLength)

[6, 11, 14, 18]
49


### Creating RDD from Collections

#### Parallelized Collections
- RDDs are created by parallelizing an existing collection in your driver program or referencing a dataset in an external storage system.
- RDDs are created by taking the existing collection and passing it to SparkContext parallelize() method.

In [22]:
data=['Simplilearn', 'is', 'an', 'educational','revolution']

rdd1=sc.parallelize(data)

rdd1.take(2)

['Simplilearn', 'is']

#### Existing Data
- RDDs can be created from existing RDDs by transforming one RDD into another RDD.

In [26]:
nums=rdd1.map(lambda x:len(x))

In [27]:
nums.collect()

[11, 2, 2, 11, 10]

#### External Data
- In Spark, a dataset can be created from any other dataset. The other dataset must be supported by Hadoop, including the local file system, HDFS, Cassandra, HBase, and many more.
- Data frame reader interface can be used to load dataset from an external storage system in the following formats:

In [32]:
spark=SparkSession(sc)

In [39]:
salaries=spark.read.csv('salaries.csv',inferSchema=True,header=True).rdd
salaries.take(5)

[Row(company='google', job='sales executive', degree='bachelors', salary_more_then_100k=0),
 Row(company='google', job='sales executive', degree='masters', salary_more_then_100k=0),
 Row(company='google', job='business manager', degree='bachelors', salary_more_then_100k=1),
 Row(company='google', job='business manager', degree='masters', salary_more_then_100k=1),
 Row(company='google', job='computer programmer', degree='bachelors', salary_more_then_100k=0)]

#### Creating RDD from a Text File
- To create a file-based RDD, you can use the command SparkContext.textFile or sc.textfile, and pass one or more file names.

In [44]:
txt=sc.textFile('example.txt')
txt.collect()

['first ', 'second line', 'the third line', 'then a fourth line']

In [45]:
txt.count()

4